# Credit Risk Resampling Techniques

In [75]:
import warnings
warnings.filterwarnings('ignore')

In [76]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [77]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [78]:
# define the data path
file_path = Path('LoanStats_2019Q1.csv')

# load the data - "skiprows=1)[:-2]" skips the first rows and the last two ones.
df = pd.read_csv(file_path, skiprows=1)[:-2]

# .loc is locating all the columns and all the rows that are also assigned to the variable column, and making a copy of it, so now we have a dataframe with the skipped rows that are not actual data.
df = df.loc[:, columns].copy()
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,20000.0,17.19%,499.10,RENT,47000.0,Source Verified,Mar-2019,Issued,n,14.02,...,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,N,N
1,21225.0,14.74%,502.05,MORTGAGE,225000.0,Not Verified,Mar-2019,Issued,n,16.80,...,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,N,N
2,5000.0,17.97%,180.69,MORTGAGE,62000.0,Not Verified,Mar-2019,Issued,n,19.82,...,66.7,50.0,0.0,0.0,255738.0,31615.0,9400.0,39938.0,N,N
3,20000.0,8.19%,628.49,MORTGAGE,200000.0,Not Verified,Mar-2019,Issued,n,22.66,...,95.7,22.2,0.0,0.0,448069.0,84744.0,49400.0,105180.0,N,N
4,12000.0,15.57%,289.09,MORTGAGE,49000.0,Source Verified,Mar-2019,Issued,n,13.47,...,86.1,14.3,0.0,0.0,189260.0,106025.0,24400.0,68860.0,N,N


In [79]:
# first we are dropping the columns that are completely empty, then we look at the rows and drop the rows that have at least one null value.
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()
df.head(2)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,20000.0,17.19%,499.10,RENT,47000.0,Source Verified,Mar-2019,Issued,n,14.02,...,98.0,12.5,0.0,0.0,75824.0,31546.0,33800.0,21524.0,N,N
1,21225.0,14.74%,502.05,MORTGAGE,225000.0,Not Verified,Mar-2019,Issued,n,16.80,...,100.0,50.0,1.0,0.0,747075.0,209426.0,53500.0,128175.0,N,N


In [80]:
# Remove the `Issued` loan status (we only want to look at loans that haven't been issued already, so we are filtering on the 'loan_status' column to display only the ones values that are not equal to 'issued').
issued_mask = df['loan_status'] != 'Issued'

df = df.loc[issued_mask]

df.head(2)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
93,10500.0,17.19%,375.35,RENT,66000.0,Source Verified,Mar-2019,Current,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
99,25000.0,20.00%,929.09,MORTGAGE,105000.0,Verified,Mar-2019,Current,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N


In [81]:
# check datatype - we can see that it is an object.
df.dtypes

loan_amnt                     float64
int_rate                       object
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [82]:
# convert interest rate to numerical - we are replacing the '%' by a nothing '',
# and using .astype to convert the object to a numeric type 'float' and divide the results by 100.
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

In [83]:
# check datatype - now we see that it tis a float, ans that's how we need it to be.
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [84]:
# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head(2)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N


# Split the Data into Training and Testing

In [85]:
# Create our features (.get_dummies() is converting all strings values into numeric ones,
# by removing the whole columns and then adding one column per each new numeric value at the end, 
# and dropping the 'loan_status' column since this is the one to be used as the target.
X = pd.get_dummies(df.drop(columns='loan_status'))

# Create our target - the target is the outcome we want to check. 
# For this porject, we want to check if the loan is more likely going to be be high_rick or low_risk.
y = df['loan_status']



In [86]:
# generate the summary statistics - here we are using '.describe' to display the summary statistic of X, 
# which represents the features/columns that we are using for analysis and prediction.
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [87]:
X.dtypes

loan_amnt                      float64
int_rate                       float64
installment                    float64
annual_inc                     float64
dti                            float64
                                ...   
next_pymnt_d_May-2019            uint8
application_type_Individual      uint8
application_type_Joint App       uint8
hardship_flag_N                  uint8
debt_settlement_flag_N           uint8
Length: 95, dtype: object

In [88]:
# Check the balance of our target values - we can see that we have a lot more low_risk values values than high_risk, 
# which makes sense for a bank to have this natural imbalance.
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [89]:
# import train_test_split 
from sklearn.model_selection import train_test_split

# here we are defining the train and test for both the features 'X' and the target 'y', 
# then in parenthesis we call the X and y variables and choose which random_state to use 
# (which is important to be the same one throughout the analysis to ensure that the random numbers are generated in the same order).
# The "stratify=y" ensures that both the train and test sets have the proportion of examples in each class that is present in the provided “y” array.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify= y)
X_train.shape

(51612, 95)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

One way to fight imbalance data is to generate new samples in the minority classes. The most naive strategy is to generate new samples by randomly sampling with replacement of the currently available samples. The RandomOverSampler offers such a scheme.

In [90]:
# Resample the training data with the RandomOversampler

# implement random oversampling
from imblearn.over_sampling import RandomOverSampler

# Here we are declaring the variable ros and assigning it to the RandomOverSamples method with the "random_state=1".
# The RandomOverSamples method is used to over-sample the minority class by picking samples at random with replacement.
ros = RandomOverSampler(random_state=1)

# To the "ros" variable, we apply the "fit_resample" method which basically resample's the dataset. 
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#Then we count the y_resampled to make sure that now "lo+risk" and "high_risk" have the same amount of values for the oversampling analysis purpose.
Counter(y_resampled)



Counter({'low_risk': 51352, 'high_risk': 51352})

In [91]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

# create the model - "Logisticregression" is a classification algorithm rather than regression algorithm.
# Based on a given set of independent variables, it is used to estimate discrete value (0 or 1, yes/no, true/false).
# solver='lbfgs' — Stands for Limited-memory Broyden–Fletcher–Goldfarb–Shanno. 
# It approximates the second derivative matrix updates with gradient evaluations. It stores only the last few updates, so it saves memory.
model = LogisticRegression(solver='lbfgs', random_state=1)

# Train the model - fitting is equal to training. Then, after it is trained, the model can be used to make predictions, usually with a .predict() method call.
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [92]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

# Predict - The predict() function accepts only a single argument which is usually the data to be tested, in this case X-test.
y_pred = model.predict(X_test)

# The balance_accuracy_score is defined as the average of recall obtained on each class.
ros_accuracy = balanced_accuracy_score(y_test, y_pred)
ros_accuracy

0.6456130066757718

• "y_pred" represents the Estimated targets as returned by a classifier.

In [93]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

# define the y_pred
y_pred = model.predict(X_test)

# generate the confusion matrix by using the y_test and y_pred as the arguments.
# The matrix compares the actual target values with those predicted by the machine learning model.
confusion_matrix(y_test, y_pred)

array([[   53,    34],
       [ 5443, 11675]])

In [94]:
# Print the imbalanced classification report using "y_test" and "y_pred" as their arguments.
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.68      0.02      0.64      0.41        87
   low_risk       1.00      0.68      0.61      0.81      0.64      0.42     17118

avg / total       0.99      0.68      0.61      0.81      0.64      0.42     17205



#### Explanation of the classification report

• Precision will give us the proportion of positive identifications that were indeed correct.

• The sensitivity/recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The sensitivity quantifies the ability to avoid false negatives_[1]. In other words, recall will determine the proportion of real positives that were correctly identified.

• The specificity is the ratio tn / (tn + fp) where tn is the number of true negatives and fn the number of false negatives. The specificity quantifies the ability to avoid false positives_[1].


• F1 Score is a metric that is useful when we need to seek a balance between precision and recall.

• geo - The geometric mean (G-mean) is the root of the product of class-wise sensitivity. This measure tries to maximize the accuracy on each of the classes while keeping these accuracies balanced. For binary classification G-mean is the squared root of the product of the sensitivity and specificity. For multi-class problems it is a higher root of the product of sensitivity for each class. The best value is 1 and the worst value is 0. 

• iba -  index balanced accuracy of the geometric mean.

• The support is the number of occurrences of each class in y_true.

#### How to analyze:

Since our objective is to minimize company loss, predicting the risk of client default, a good recall rate is desirable because we want to identify the maximum amount of clients that are indeed prone to stop paying their debts, thus, we are pursuing a small number of False Negatives.

analysis:




### SMOTE Oversampling

SMOTE (Synthetic Minority Oversampling Technique) works by randomly picking a point from the minority class and computing the k-nearest neighbors for this point. The synthetic points are added between the chosen point and its neighbors.

In [95]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
# Count the "y_resmapled" to make sure both "high_risk" and "low_risk" now have the same amount of values, which means they are balanced.
Counter(y_resampled)

Counter({'low_risk': 51352, 'high_risk': 51352})

In [96]:
# Train the Logistic Regression model using the resampled data

# create the model - "Logisticregression" is a classification algorithm rather than regression algorithm.
# Based on a given set of independent variables, it is used to estimate discrete value (0 or 1, yes/no, true/false).
# solver='lbfgs' — Stands for Limited-memory Broyden–Fletcher–Goldfarb–Shanno. 
# It approximates the second derivative matrix updates with gradient evaluations. It stores only the last few updates, so it saves memory.
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [97]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
smote_accuracy = balanced_accuracy_score(y_test, y_pred)
smote_accuracy

0.6234433606890912

In [98]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,53,34
Actual low_risk,6202,10916


In [99]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.61      0.64      0.02      0.62      0.39        87
   low_risk       1.00      0.64      0.61      0.78      0.62      0.39     17118

avg / total       0.99      0.64      0.61      0.77      0.62      0.39     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [100]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 260, 'low_risk': 260})

In [101]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [102]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
undersampling_accuracy = balanced_accuracy_score(y_test, y_pred)
undersampling_accuracy

0.5177862786097313

In [103]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,50,37
Actual low_risk,9229,7889


In [104]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.57      0.46      0.01      0.51      0.27        87
   low_risk       1.00      0.46      0.57      0.63      0.51      0.26     17118

avg / total       0.99      0.46      0.57      0.63      0.51      0.26     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [105]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# Count the resampled classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'high_risk': 68458, 'low_risk': 62022})

In [106]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [107]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
smoteenn_accuracy = balanced_accuracy_score(y_test, y_pred)
smoteenn_accuracy 

0.6531287896185101

In [108]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])

cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,60,27
Actual low_risk,6563,10555


In [109]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.62      0.02      0.65      0.43        87
   low_risk       1.00      0.62      0.69      0.76      0.65      0.42     17118

avg / total       0.99      0.62      0.69      0.76      0.65      0.42     17205



### Accuracy Score Results

In [110]:
pd.DataFrame({"Model":["RandomOverSampler","SMOTE Oversampling","CC Undersampling","SMOTEENN Combination"],
            "Balanced Accuracy":[ros_accuracy, smote_accuracy, undersampling_accuracy, smoteenn_accuracy]})



,Model,Balanced Accuracy
0,RandomOverSampler,0.645613
1,SMOTE Oversampling,0.623443
2,CC Undersampling,0.517786
3,SMOTEENN Combination,0.653129
